In [1]:
import torch
from transformers import XLMRobertaTokenizer, XLMRobertaForSequenceClassification
import numpy as np
import pandas as pd
from tqdm import tqdm
from scipy.stats import pearsonr
import random

In [2]:
val_data = pd.read_csv('data/messages_dev_features_ready_for_WS_2022.tsv', sep='\t')

In [3]:
model_path = 'best_model.pt'
saved_model = torch.load(model_path)

model_path2 = 'dis_model_new.pt'
saved_model2 = torch.load(model_path2)

In [4]:
tokenizer = XLMRobertaTokenizer.from_pretrained('xlm-roberta-base')
model = XLMRobertaForSequenceClassification.from_pretrained('xlm-roberta-base', num_labels=1)
model2 = XLMRobertaForSequenceClassification.from_pretrained('xlm-roberta-base', num_labels=1)

Some weights of the model checkpoint at xlm-roberta-base were not used when initializing XLMRobertaForSequenceClassification: ['lm_head.layer_norm.bias', 'lm_head.decoder.weight', 'lm_head.bias', 'roberta.pooler.dense.bias', 'lm_head.dense.weight', 'roberta.pooler.dense.weight', 'lm_head.layer_norm.weight', 'lm_head.dense.bias']
- This IS expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLMRobertaForSequenceClassification were not initialized from the model checkpoint at xlm-roberta-base and are newly initialized: ['classifier.out_p

In [5]:
model.load_state_dict(saved_model)
model2.load_state_dict(saved_model2)

<All keys matched successfully>

In [6]:
model.eval()
model2.eval()
print("")

In [7]:
def predict_score(sentences):
  inputs = tokenizer(sentences, padding=True, truncation=True, max_length=512, return_tensors='pt')
  input_ids = inputs['input_ids']
  attention_mask = inputs['attention_mask']
  with torch.no_grad():
    outputs = model(input_ids, attention_mask=attention_mask)
    outputs2 = model2(input_ids, attention_mask=attention_mask)
  predictions_emp = outputs.logits.squeeze(1).detach().cpu().numpy()
  predictions_dis = outputs2.logits.squeeze(1).detach().cpu().numpy()
  return predictions_emp,predictions_dis

In [8]:
acu_emp = val_data['empathy'].tolist()
acu_dis = val_data['distress'].tolist()

In [9]:
sen_org = val_data['essay'].tolist()

In [10]:
type(sen_org)

list

In [11]:
pred_emp_org, pred_dis_org = predict_score(sen_org)

In [12]:
pred_emp_org.tolist()
print(pred_emp_org[0])

3.8101296


In [13]:
pred_dis_org.tolist()
print(pred_dis_org[0])

3.770312


In [14]:
corr_coef, p_value = pearsonr(pred_emp_org, acu_emp)

print("Pearson correlation coefficient emp stack:", corr_coef)

Pearson correlation coefficient emp stack: 0.2845973035742949


In [15]:
corr_coef, p_value = pearsonr(pred_dis_org, acu_dis)

print("Pearson correlation coefficient dis stack:", corr_coef)

Pearson correlation coefficient dis stack: 0.0614218996625809


In [16]:
def get_positive_score(l_aug,org,aug):
  wrong_list = []
  for i in range(len(org)):
    if (abs(org[i]-aug[i])>0.06):
      wrong_list.append(i)
  print("Error score:",len(wrong_list)/len(org))
  if(len(wrong_list)<=5):
        random_five = wrong_list
  else:
    random_five = random.sample(wrong_list, 5)
  for j in random_five:
    print(val_data['essay'].values[j])
    print("score:",org[j])
    print(l_aug['essay'].values[j])
    print("score:",aug[j])

In [17]:
def get_scores(l_aug,pred_emp_org, acu_emp, pred_dis_org, acu_dis,pred_emp_aug, pred_dis_aug):
  
  corr_coef_f_aug, p_value = pearsonr(pred_emp_aug, acu_emp)
  print("Pearson Fution empathy:", corr_coef_f_aug)

  corr_coef_s_a, p_value = pearsonr(pred_dis_aug, acu_dis)
  print("Pearson Stack distress:", corr_coef_s_a)

  print("Empathy:")
  get_positive_score(l_aug,pred_emp_org,pred_emp_aug)

  print("Distress:")
  get_positive_score(l_aug,pred_dis_org,pred_dis_aug)

In [18]:
data_bn = pd.read_csv('t_data/test_data_bn.tsv', sep='\t')
sen_bn = data_bn['essay'].tolist()
pred_emp_bn, pred_dis_bn = predict_score(sen_bn)
get_scores(data_bn,pred_emp_org, acu_emp, pred_dis_org, acu_dis,pred_emp_bn, pred_dis_bn)

Pearson Fution empathy: 0.19098410080074382
Pearson Stack distress: 0.12432172449708714
Empathy:
Error score: 0.1
A little boy had fallen out of window because he was not being watched. It is important that children area always watched because bad things can happen in a split second and its important that the parents always have eyes on their kids because it's that one second that it takes for the kids to get hurt or worse die I hope these people learned an importat listen and that the child is  ok and he can live a long life.
score: 3.7182043
একটি ছোট ছেলে জানালা দিয়ে পড়ে গিয়েছিল কারণ তাকে দেখা যাচ্ছিল না। এটা গুরুত্বপূর্ণ যে বাচ্চাদের এলাকা সবসময় দেখেছে কারণ খারাপ জিনিসগুলি একটি বিভক্ত সেকেন্ডে ঘটতে পারে এবং এটি গুরুত্বপূর্ণ যে পিতামাতারা সর্বদা তাদের বাচ্চাদের দিকে নজর রাখেন কারণ এটি সেই এক সেকেন্ড যে বাচ্চাদের আঘাত পেতে বা খারাপ হতে লাগে আমি আশা করি এই লোকেরা একটি importat শুনতে শিখেছি এবং শিশু ঠিক আছে এবং সে একটি দীর্ঘ জীবন বাঁচতে পারে.
score: 3.786568
Dear friend, I have just

In [19]:
data_de = pd.read_csv('t_data/test_data_de.tsv', sep='\t')
sen_de = data_de['essay'].tolist()
pred_emp_de, pred_dis_de = predict_score(sen_de)
get_scores(data_de,pred_emp_org, acu_emp, pred_dis_org, acu_dis,pred_emp_de, pred_dis_de)

Pearson Fution empathy: 0.2720828437328491
Pearson Stack distress: 0.0025543147594948414
Empathy:
Error score: 0.03333333333333333
I cannot imagine what is going through those parent's minds.  To not be able to protect your kids, sending them to school believing that they would be safe.  It's every parent's worst nightmare.  I don't know what would be worse, having hope that the kids are still alive or never having closure.  And how scared the kids must have been!  I don't know how anyone could hurt children.  They can't have any conscience or heart.
score: 3.733577
Ich kann mir nicht vorstellen, was in den Köpfen dieser Eltern vorgeht. Ihre Kinder nicht beschützen zu können, sie in die Schule zu schicken in dem Glauben, dass sie in Sicherheit wären. Es ist der schlimmste Albtraum aller Eltern. Ich weiß nicht, was schlimmer wäre, die Hoffnung zu haben, dass die Kinder noch am Leben sind, oder nie eine Schließung zu haben. Und wie verängstigt müssen die Kinder gewesen sein! Ich weiß nic

In [20]:
data_el = pd.read_csv('t_data/test_data_el.tsv', sep='\t')
sen_el = data_el['essay'].tolist()
pred_emp_el, pred_dis_el = predict_score(sen_el)
get_scores(data_el,pred_emp_org, acu_emp, pred_dis_org, acu_dis,pred_emp_el, pred_dis_el)

Pearson Fution empathy: 0.2555067317778482
Pearson Stack distress: 0.038610883305893254
Empathy:
Error score: 0.040740740740740744
I just read an article about a democrat running for office. It talked about "if Hillary won the election" How can people be so ignorant. If Hillary won the election America would be in a much worse situation than it is currently. It just amazes me what they focus their elections on. Ok, guns kill people...whatever Democrats kill babies. Why has nobody been bright enough to realize that? I think the country is slowly becoming retarded.
score: 3.715505
Μόλις διάβασα ένα άρθρο για έναν δημοκράτη που διεκδικεί το αξίωμα. Μίλησε για το "αν η Χίλαρι κέρδιζε τις εκλογές" Πώς μπορεί οι άνθρωποι να είναι τόσο αδαείς. Εάν η Χίλαρι κέρδιζε τις εκλογές, η Αμερική θα βρισκόταν σε πολύ χειρότερη κατάσταση από αυτή που βρίσκεται σήμερα. Απλώς με εκπλήσσει σε τι επικεντρώνουν τις εκλογές τους. Εντάξει, τα όπλα σκοτώνουν ανθρώπους...ό,τι οι Δημοκρατικοί σκοτώνουν μωρά. Γιατ

In [21]:
data_hi = pd.read_csv('t_data/test_data_hi.tsv', sep='\t')
sen_hi = data_hi['essay'].tolist()
pred_emp_hi, pred_dis_hi = predict_score(sen_hi)
get_scores(data_hi,pred_emp_org, acu_emp, pred_dis_org, acu_dis,pred_emp_hi, pred_dis_hi)

Pearson Fution empathy: 0.21890880409974495
Pearson Stack distress: 0.09899184397240737
Empathy:
Error score: 0.15185185185185185
That was a very long read. My take is the usual dealing with businesses like this. Any research not done by a neutral third party can't be relied on because it could be biased. Concern is legitimate in this case before starting the project. The explanation about what the toxin can do felt a little vague for most of the paper.
score: 3.7173479
वह बहुत लंबा पढ़ा गया था। मैं इस तरह के व्यवसायों के साथ सामान्य व्यवहार करना चाहता हूं। तटस्थ तृतीय पक्ष द्वारा नहीं किए गए किसी भी शोध पर भरोसा नहीं किया जा सकता क्योंकि यह पक्षपातपूर्ण हो सकता है। इस मामले में परियोजना शुरू करने से पहले चिंता जायज है। अधिकांश पेपर के लिए विष क्या कर सकता है, इसके बारे में स्पष्टीकरण थोड़ा अस्पष्ट लगा।
score: 3.7815912
Extra extra fatties whine about not being treated like real people. Wah people expect us to lose weight before they'll take us seriously. Whats up with modern culture a

In [22]:
data_nel = pd.read_csv('t_data/test_data_nel.tsv', sep='\t')
sen_nel = data_nel['essay'].tolist()
pred_emp_nel, pred_dis_nel = predict_score(sen_nel)
get_scores(data_nel,pred_emp_org, acu_emp, pred_dis_org, acu_dis,pred_emp_nel, pred_dis_nel)

Pearson Fution empathy: 0.22677398312374325
Pearson Stack distress: 0.036699628877294915
Empathy:
Error score: 0.2111111111111111
Timeshare owners have long needed to be dealt with in this manner. I feel like overall there are quite a bit of them that are just not good people overall that control these timeshare contracts. Trying to relinquish them is a pain in the butt and it should be nipped or we cant do things right in my honest opinion.
score: 3.7107227
टाइमशेयर मालिकहरूलाई यस तरिकामा व्यवहार गर्न लामो समय आवश्यक छ। मलाई लाग्छ कि समग्रमा तिनीहरूमध्ये धेरै थोरै छन् जुन यी टाइमसेयर अनुबंधहरू नियन्त्रण गर्ने समग्रमा राम्रो व्यक्तिहरू छैनन्। तिनीहरूलाई त्याग्ने प्रयास गर्नु नितम्बको दुखाइ हो र यसलाई निपाउनु पर्छ वा हामी मेरो इमानदार विचारमा चीजहरू गर्न सक्दैनौं।
score: 3.7751215
I just read this story about these two idiot divers. It's such a pointless and selfish hobby. Oooh, I get to see a dark cave. That's what's drones are for. They've ruined their families lives because they want

In [23]:
data_sw = pd.read_csv('t_data/test_data_sw.tsv', sep='\t')
sen_sw = data_sw['essay'].tolist()
pred_emp_sw, pred_dis_sw = predict_score(sen_sw)
get_scores(data_sw,pred_emp_org, acu_emp, pred_dis_org, acu_dis,pred_emp_sw, pred_dis_sw)

Pearson Fution empathy: 0.2530193939842204
Pearson Stack distress: -0.016110945711100003
Empathy:
Error score: 0.040740740740740744
Another attack on a mall - while no one was actually hurt, this is just an ongoing trend that concerns me so much.  It's gottten to where I Feel like nowhere we go is safe anymore.  I don't like going to crowded places anymore or anywhere that there are a lot of people.  It seems like it's more likely to happen there plus it's always some crazy immigrant.
score: 3.708113
Shambulio lingine kwenye duka la maduka - wakati hakuna mtu aliyejeruhiwa, huu ni mtindo unaoendelea ambao unanihusu sana. Imefika ambapo Ninahisi kama hakuna mahali tunapoenda ni salama tena. Sipendi tena kwenda sehemu zenye watu wengi au mahali popote ambapo kuna watu wengi. Inaonekana kuna uwezekano mkubwa wa kutokea huko pamoja na kuwa kila mara ni wahamiaji wazimu.
score: 3.7800138
We're really screwed about climate change, aren't we? I can't believe that there are real people that ma

In [24]:
data_tel = pd.read_csv('t_data/test_data_tel.tsv', sep='\t')
sen_tel = data_tel['essay'].tolist()
pred_emp_tel, pred_dis_tel = predict_score(sen_tel)
get_scores(data_tel,pred_emp_org, acu_emp, pred_dis_org, acu_dis,pred_emp_tel, pred_dis_tel)

Pearson Fution empathy: 0.2229932063310027
Pearson Stack distress: 0.1504100906256847
Empathy:
Error score: 0.18888888888888888
I just read that a chain of for profit schools got shut down. Good riddance honestly. Things like education, prisons, healthcare, etc. shouldn't be for profit. They were probably taking advantage of young people, so they deserve to be shut down. They were using public funds on top of that. Like why should the school receive tax payer money if they're profiting?
score: 3.720906
లాభాల కోసం పాఠశాలలు మూసివేయబడిందని నేను ఇప్పుడే చదివాను. నిజాయితీగా మంచి రిడాన్స్. విద్య, జైళ్లు, వైద్యం తదితర అంశాలు లాభాపేక్షతో ఉండకూడదు. వారు బహుశా యువకుల ప్రయోజనాన్ని పొందుతున్నారు, కాబట్టి వారు మూసివేయబడటానికి అర్హులు. పైగా ప్రజాధనాన్ని వినియోగించుకున్నారు. లాభదాయకంగా ఉంటే పాఠశాల పన్ను చెల్లింపుదారుల డబ్బును ఎందుకు స్వీకరించాలి?
score: 3.7979488
So I just read an article about a new lawsuit that is happening. There is a group of women who are suing L'Oréal. Apparently after using on